In [ ]:
import pandas as pd
import folium
from folium import plugins
import numpy as np

# CSV 파일에서 CCTV 데이터 읽기
def load_cctv_data(csv_file_path):
    """
    CSV 파일에서 CCTV 데이터를 읽어오는 함수
    
    Parameters:
    csv_file_path (str): CSV 파일 경로
    
    Returns:
    pandas.DataFrame: CCTV 데이터
    """
    try:
        # CSV 파일 읽기 (인코딩 문제 대비)
        df = pd.read_csv(csv_file_path, encoding='utf-8-sig')
        
        df= df.rename(columns={'위도': 'latitude', '경도': 'longitude'})
        
        # 결측값 제거
        df = df.dropna(subset=['latitude', 'longitude'])
        
        # 좌표 범위 확인 (부산 지역 범위)
        busan_lat_range = (33, 36)
        busan_lng_range = (120.8, 131.3)
        
        df = df[
            (df['latitude'] >= busan_lat_range[0]) & 
            (df['latitude'] <= busan_lat_range[1]) &
            (df['longitude'] >= busan_lng_range[0]) & 
            (df['longitude'] <= busan_lng_range[1])
        ]
        
        print(f"총 {len(df)}개의 CCTV 데이터를 로드했습니다.")
        return df
        
    except Exception as e:
        print(f"데이터 로딩 중 오류 발생: {e}")
        return None

def create_busan_cctv_dots(df, save_path='busan_boandeong.html'):
    """
    부산 CCTV 분포를 점으로 표시
    
    Parameters:
    df (pandas.DataFrame): CCTV 데이터
    save_path (str): 저장할 HTML 파일 경로
    
    Returns:
    folium.Map: 생성된 지도 객체
    """
    
    # 부산 중심 좌표 (부산시청 기준)
    busan_center = [35.1796, 129.0756]
    
    # 기본 지도 생성
    m = folium.Map(
        location=busan_center,
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # CCTV 위치를 작은 원점으로 표시
    for idx, row in df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=1,  # 점 크기
            popup=f"boandeong #{idx+1}",
            color='purple',  # 테두리 색상
            fillColor='purple',  # 내부 색상
            fillOpacity=0.8,
            weight=1  # 테두리 두께
        ).add_to(m)
    
    # 통계 정보를 담은 HTML 추가
    stats_html = f"""
    <div style='position: fixed; 
                top: 10px; right: 10px; width: 200px; height: 70px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px; opacity: 0.9;'>
    <p><b>부산시 보안등</b></p>
    <p>총 보안등 수: {len(df)}개</p>
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(stats_html))
    
    # 지도 저장
    m.save(save_path)
    print(f"점 지도가 '{save_path}'에 저장되었습니다.")
    
    return m

# 메인 실행 함수
def main():
    # CSV 파일 경로 (실제 파일 경로로 수정하세요)
    csv_file_path = '보안등정보_위경도포함.csv'
    
    # 데이터 로드
    df = load_cctv_data(csv_file_path)
    
    if df is not None:
        # 데이터 기본 정보 출력
        print("\n=== 데이터 정보 ===")
        print(f"총 데이터 수: {len(df)}")
        print(f"위도 범위: {df['latitude'].min():.6f} ~ {df['latitude'].max():.6f}")
        print(f"경도 범위: {df['longitude'].min():.6f} ~ {df['longitude'].max():.6f}")
        
        # 점 지도 생성
        map_obj = create_busan_cctv_dots(df)
        
        print("\n=== 점 지도 안내 ===")
        print("- 빨간 점: 각 CCTV 위치")
        print("- 점 클릭: CCTV 번호 확인 가능")
        print("- 확대/축소로 더 자세한 분포 확인 가능")
    
    else:
        print("데이터를 로드할 수 없습니다. CSV 파일 경로와 컬럼명을 확인하세요.")
        
        # 샘플 CSV 형식 안내
        print("\n=== 예상 CSV 파일 형식 ===")
        print("latitude,longitude")
        print("35.1587,129.0607")
        print("35.1796,129.0756")
        print("...")

if __name__ == "__main__":
    main()

# 간단한 실행 함수들
def create_simple_dots(csv_file_path):
    """가장 간단한 점 지도 생성"""
    df = load_cctv_data(csv_file_path)
    if df is None:
        return
    
    # 간단한 점 지도
    m = folium.Map(location=[35.1796, 129.0756], zoom_start=11)
    
    for idx, row in df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=1,
            color='red',
            fillColor='red',
            fillOpacity=0.8
        ).add_to(m)
    
    m.save('simple_dots.html')
    print("간단한 점 지도가 'simple_dots.html'에 저장되었습니다.")

def create_simple_heatmap(csv_file_path):
    """가장 간단한 히트맵 생성"""
    df = load_cctv_data(csv_file_path)
    if df is None:
        return
    
    # 간단한 히트맵
    m = folium.Map(location=[35.1796, 129.0756], zoom_start=11)
    
    heat_data = [[row['latitude'], row['longitude']] for idx, row in df.iterrows()]
    plugins.HeatMap(heat_data).add_to(m)
    
    m.save('simple_heatmap.html')
    print("간단한 히트맵이 'simple_heatmap.html'에 저장되었습니다.")

# 히트맵과 점 지도 모두 생성하는 함수
def create_both_maps(csv_file_path):
    """히트맵과 점 지도 모두 생성"""
    df = load_cctv_data(csv_file_path)
    if df is None:
        return
    
    print("점 지도 생성 중...")
    create_busan_cctv_dots(df, 'busan_cctv_dots.html')
    
    print("히트맵 생성 중...")
    # 히트맵 함수 재정의
    m = folium.Map(location=[35.1796, 129.0756], zoom_start=11)
    heat_data = [[row['latitude'], row['longitude']] for idx, row in df.iterrows()]
    plugins.HeatMap(heat_data, radius=20, blur=15).add_to(m)
    m.save('busan_cctv_heatmap.html')
    
    print("두 지도 모두 생성 완료!")

총 72352개의 CCTV 데이터를 로드했습니다.

=== 데이터 정보 ===
총 데이터 수: 72352
위도 범위: 35.006082 ~ 35.385479
경도 범위: 128.805626 ~ 129.291306
점 지도가 'busan_boandeong.html'에 저장되었습니다.

=== 점 지도 안내 ===
- 빨간 점: 각 CCTV 위치
- 점 클릭: CCTV 번호 확인 가능
- 확대/축소로 더 자세한 분포 확인 가능
